In [1]:
import pandas as pd
import numpy as np

$Y = median\_house\_value \Rightarrow$ Verkliga värden, målvariabel

$X = designmatris \Rightarrow$ Alla regressorer, shape

$\hat{Y} = \hat{\beta}_0 + \hat{\beta}_1X_1 + \hat{\beta}_2X_2 + \hat{\beta}_3X_3 ... + \hat{\beta}_dX_d \Rightarrow$ Modellens prediktion för varje observation

$e_i = Y_i - \hat{Y}_i \Rightarrow$ Residual för observation $i$

$SSE = \sum_{i=1}^{n} e_i^2 \Rightarrow$ Sum of Squared Errors - totala felet för modellen

$S_{yy} = \sum(Y_i - \bar{Y})^2 = \sum{Y_{i}}^2 - \frac{(\sum{Y_i})^2}{n} \Rightarrow$ Totala variationen i Y

$SSR = S_{yy} - SSE \Rightarrow$ Regression Sum of Squares - variation som modellen förklarar

$R^2 = \frac{SSR}{S_{yy}} \Rightarrow$ Bestämningskoefficient - hur mycket av Y som modellen förklarar

$\hat{\sigma}^2 = \frac{SSE}{n-d-1} \Rightarrow$ Estimering av residualvarians

$RMSE = \sqrt{\frac{SSE}{n-d-1}} \Rightarrow$ Root Mean Squared Error - spridning i samma enhet som Y (använder $\sqrt{\frac{SSE}{n-d-1}}$ istället för $\sqrt{\frac{SSE}{n}}$ då den senare används i större utsträckning inom ML och uppgiften skulle vara en statistisk analys)

$\hat{\beta} = (X^TX)^{-1}X^TY \Rightarrow$ OLS-koefficienter - beräkning av alla $\beta$ från datan

$C = \hat{\sigma}^2(X^TX)^{-1} \Rightarrow$ Varians och kovarians för $\hat{\beta}$, används i t-test och konfidensintervall

$t_i = \frac{\hat{\beta}_i}{\hat{\sigma}\sqrt{c_{ii}}} \Rightarrow$ t-statistik - Testar om en enskild koefficient skiljer sig signifikant från 0

$F = \frac{SSR/d}{\hat{\sigma}^2} \Rightarrow$ Testar om alla regressorer tillsammans är signifikanta

$\hat{\beta}_i \pm t_{\alpha/2, n-d-1}\hat{\sigma}\sqrt{c_{ii}} \Rightarrow$ Intervallet för $\beta$ med viss säkerhetsnivå

$r = \frac{Cov(X_a,X_b)}{\sqrt{Var(X_a)Var(X_b)}} \Rightarrow$ Pearson $r$ - Mäta linjärt beroende mellan två regressorer

$d$ = antal regressorer  
$n$ = antal observationer  
$c_{ii}$ = diagonalelement i varians-kovariansmatrisen $C$  
Alla formler bygger direkt på residualerna $e_i$ och $X$-matrisen

Datasetet läses in som en dataframe och aggregeras med funktionen aggregate från linreg.py till en observation per geografisk punkt (longitude, latitude), då flera koordinater förekommer mer än en gång. Additiva variabler summeras medan nivåvariabler ersätts av det nya medelvärdet. Den kategoriska variabeln ocean_proximity var konstant inom varje grupp vilket även verifierades och möjliggjorde oförändrad inkludering efter aggregering där första värdet per grupp valdes, detta valet påverkade dock inget då alla grupper bara hade samma värden.

Därefter one-hot-encodas dataframen med funktionen prepare_features från linreg.py där första kategorin droppas, i detta fallet <1h from ocean som var den näst mest förekommande på drygt 1/3 av grupperna.

In [2]:
from linreg import LinearRegression, aggregate, prepare_features, build_X_Y
housing = pd.read_csv('housing.csv')
aggregated = aggregate(housing)
aggregated, X_cols = prepare_features(aggregated, cat_cols=['ocean_proximity'])

kontroll av kategoriska variabler

one hot-encoding

bygger X och Y samt lägger till intercept-kolumnen $\beta_0$

In [3]:
X, Y = build_X_Y(aggregated, X_cols)
n = X.shape[0]
d = X.shape[1] - 1


kör OLS-formeln samt beräkning av grundläggande statistik så som $\hat{\sigma}^2$ och $RMSE$

In [4]:
model = LinearRegression(X, Y)
print(f'modellens beta-värden {model.b_est}')         # modellens beta-värden
print(f'modellens prediktioner {model.Y_est}')        # modellens prediktioner
print(f'modellens residualer {model.residuals}')      # residualer
print(f'modellens SSE {model.SSE}')                   # sum of squared errors
print(f'modellens varians {model.variance}')          # modellens varians
print(f'modellens STD {model.STD}')                   # modellens standardavvikelse
print(f'modellens RMSE {model.RMSE}')                 # modellens RMSE (nästan samma som denna modellens STD p.g.a. stort n-tal)

modellens beta-värden [-1.68425122e+06 -2.02550076e+04 -1.95460964e+04  8.07678211e+02
  3.89702735e+04 -8.57665947e-01  3.47044243e+01 -2.26211482e+01
  3.64048403e+01 -4.35143715e+04  1.70054470e+05 -8.41692780e+03
  1.15252233e+04]
modellens prediktioner [213515.43665521 125344.37620943 163615.86977736 ...  23321.2310764
  55439.32646054  23559.63880305]
modellens residualer [-118915.43665521  -39544.37620943  -60015.86977736 ...   64178.7689236
   24660.67353946   43340.36119695]
modellens SSE 49231258393954.016
modellens varians 3914388041.1826363
modellens STD 62565.07045614698
modellens RMSE 62532.76084603433


In [5]:

print(f'modellens varians-kovariansmatris {model.C}') # hur osäkra modellens koefficienter är
print(f'modellens t-statistik {model.t_stats}')       # mäter signifikans för varje koefficient
print(f'modellens p-värden {model.p_values}')         # sannolikheten att koefficienten egentligen är 0; lågt värde = signifikant

modellens varians-kovariansmatris [[ 7.86383336e+09  8.96442512e+07  8.01462962e+07  2.85655169e+05
   3.36027209e+06 -1.77760995e+03 -4.49014455e+04  5.24223588e+02
   5.52887690e+04 -9.42145134e+07  5.71009572e+07  3.63924151e+07
   3.14151041e+07]
 [ 8.96442512e+07  1.03750106e+06  9.63710658e+05  4.20897721e+03
   4.99432199e+04 -2.36227548e+01 -4.71506256e+02  1.96070388e+01
   5.92672258e+02 -1.12505473e+06  7.43550030e+05  3.17829847e+05
   3.74185841e+05]
 [ 8.01462962e+07  9.63710658e+05  9.82745770e+05  3.75659639e+03
   5.49504120e+04 -2.52015009e+01 -3.57964530e+02  4.21434174e+01
   4.54940347e+02 -1.19589176e+06  8.56377183e+05  3.54058174e+04
   3.41530438e+05]
 [ 2.85655169e+05  4.20897721e+03  3.75659639e+03  2.71212320e+03
   2.12014834e+03  4.33463270e+00  1.08322392e+01  4.50234481e-01
  -3.09996705e+01  7.30339179e+03 -3.67364570e+04 -1.61853116e+04
   2.55415595e+02]
 [ 3.36027209e+06  4.99432199e+04  5.49504120e+04  2.12014834e+03
   1.34435014e+05 -9.22193372e+0

| Index | Variabel | p-värde | %  |
|:-----:|----------|---------|----:|
| 0 | intercept | 2.51e-79 | $\lt$ 0,01% |
| 1 | longitude | 1.16e-86 | $\lt$ 0,01% |
| 2 | latitude | 2.97e-85 | $\lt$ 0,01% |
| 3 | housing_median_age | 9.48e-54 | $\lt$ 0,01% |
| 4 | median_income | 0.00e+00 | $\ll$ 0,01% |
| 5 | total_rooms | 1.03e-01 | $\approx$ 10,3% |
| 6 | total_bedrooms | 6.72e-16 | $\lt$ 0,01% |
| 7 | population | 1.50e-198 | $\lt$ 0,01% |
| 8 | households | 7.48e-13 | $\lt$ 0,01% |
| 9 | INLAND | 1.35e-115 | $\lt$ 0,01% |
| 10 | ISLAND | 1.36e-09 | $\lt$ 0,01% |
| 11 | NEAR_BAY | 4.12e-04 | $\approx$ 0,04% |
| 12 | NEAR_OCEAN | 9.30e-10 | $\lt$ 0,01% |


Detta är då signifikansen för de olika kolumnerna där vi då kan se att median income visar extrem statistisk signifikans till median house value, total rooms har runt .103 och är alltså inte något vidare signifikant och resterande har hög signifikans där near bay är en utstickare (fortfarande väldigt hög signifikans men märkbart lägre än de andra)